In [1]:
import uuid
import random
import pandas as pd
import numpy as np
from datetime import datetime
import create_table_leanx as ctl

import values, helpers
import master_data, text_data, purchasing_doc_data

In [2]:
# text tables
for method in (
    text_data.domain_fixed_values, 
    text_data.sales_doc_types, 
    text_data.sales_organizations,
    text_data.distribution,
    text_data.sales_doc_item_categories,
    text_data.sales_doc_rejection_reasons,
    text_data.system_status,
    text_data.blocking_reasons,
    text_data.releases,
    text_data.purchasing_doc_types,
    text_data.purchasing_organizations
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])
        df.to_csv(f'data/p2p/ocpm/text/{table_name}.csv', index=False)

In [3]:
# master tables
for method in (
    master_data.users,
    master_data.customers_and_vendors, 
    master_data.plants, 
    master_data.materials, 
    master_data.material_support,
    master_data.routes,
    master_data.company_codes
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])
        df.to_csv(f'data/P2P/OCPM/master/{table_name}.csv', index=False)

C:\Users\w.falls\AppData\Local\Temp\ipykernel_2088\2547217520.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([all_cols, pd.DataFrame(v.values())])


In [4]:
def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

def get_time_consumption(start_date, planned_target_date, latest_date):
    return (latest_date - start_date) / (planned_target_date - start_date)

In [5]:
MARC = pd.read_csv('data/P2P/OCPM/master/MARC.csv')
MARA = pd.read_csv('data/P2P/OCPM/master/MARA.csv')
MAKT = pd.read_csv('data/P2P/OCPM/master/MAKT.csv')
LFB1 = pd.read_csv('data/P2P/OCPM/master/LFB1.csv')
LFA1 = pd.read_csv('data/P2P/OCPM/master/LFA1.csv')

all_prices = {}
all_has_contract_probabilities = {}
all_matnr_names = {}
all_availabilities = {}

# get price and material group by MATNR
for nr in MARA['MATNR']:
    name = MAKT[MAKT['MATNR'] == nr]['MAKTX'].values[0]
    for k, v in values.om_material_groups.items():
        for mgrp, attr in v.items():
            for mat, details in attr['materials'].items():
                if name == mat:
                    all_prices[nr] = details['price']
                    all_matnr_names[nr] = mat
                    all_has_contract_probabilities[nr] = attr['has_contract_probability']
                    all_availabilities[nr] = attr['availability']

In [6]:
def get_params():
    # basic informaiton
    company_code = random.choice(list(values.om_company_codes.keys()))
    plant = random.choice(values.om_company_codes[company_code]['plants'])
    konnr = f'SC{str(uuid.uuid4())[-15:]}'
    lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    vendor_name = LFA1[LFA1['LIFNR'] == lifnr]['NAME1'].values[0]
    purchasing_org=random.choice(values.om_plants[plant]['purchasing_orgs'])
    new_lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    payment_term = random.choice(list(LFB1[LFB1['LIFNR'] == lifnr]['ZTERM']))
    requested_by = get_user_name(0.1)

    # materials related
    all_matnrs = MARC[MARC['WERKS'] == plant]['MATNR'].unique()
    matnrs = random.sample(list(all_matnrs), min(random.randint(5, 25), len(all_matnrs)))

    # quantities
    quantity_factor = random.randint(24, 48) if values.om_plants[plant]['high_value'] else random.randint(6, 18)
    quantities = [random.randint(12, 60)*quantity_factor for temp_quantity_index in range(len(matnrs))]
    has_qty_change = True if values.om_company_codes[company_code]['incorrect_qty_prbobability'] > 0.3 else False

    # prices and contract
    prices = [all_prices[matnrs[temp_matnr_index]] for temp_matnr_index in range(len(matnrs))]
    has_contract_probabilities = [all_has_contract_probabilities[matnrs[temp_contract_index]] for temp_contract_index in range(len(matnrs))]
    item_has_contract = [True if random.random() < has_contract_probabilities[j] else False for j in range(len(has_contract_probabilities))]
    has_contract = True 
    for hc in item_has_contract:
        if hc == False:
            has_contract = False
            break
    
    # delivery status
    delivery_status = [] # early: -x | on_time: 0 | late: x
    for temp_delivery_status_index in range(len(matnrs)):
        extra_days = 0
        if all_availabilities[matnrs[temp_delivery_status_index]] < 0.3: 
            extra_days = random.randint(7, 14) # late
        elif (all_availabilities[matnrs[temp_delivery_status_index]] >= 0.3) and (all_availabilities[matnrs[temp_delivery_status_index]] < 0.8):
            extra_days = 0 # on time
        else:
            extra_days = -random.randint(3, 7)
        delivery_status.append(extra_days)
        
    # free text
    is_free_text = True if random.random() < values.om_company_codes[company_code]['free_text_pr_probability'] else False
    free_text_materials = [all_matnr_names[matnr] for matnr in all_matnrs] # TODO change with material description

    # time distribution
    pr_req_years = [2022, 2023]
    pr_req_year_probability = np.array([0.5, 0.5])
    pr_req_month_distribution = np.array([7, 6, 5, 4, 2, 2, 3, 5, 5, 6, 7, 10])
    pr_req_month_probability = pr_req_month_distribution / pr_req_month_distribution.sum(axis=0, keepdims=True)
    pr_req_month = np.random.choice(12, p=pr_req_month_probability) + 1 # because months should be in [1, 12] for datetime
    pr_req_year_probability = pr_req_year_probability / pr_req_year_probability.sum(axis=0, keepdims=True)
    pr_req_year = pr_req_years[np.random.choice(2, p=pr_req_year_probability)]

    params = {
        'matnrs': matnrs,
        'konnr': konnr,
        'lifnr': lifnr,
        'plant': plant,
        'quantities': quantities,
        'prices': prices, # considered contract and PO prices
        'company_code': company_code,
        'purchasing_org': purchasing_org,
        'payment_term': payment_term,
        'requested_by': requested_by,
        
        'item_has_contract': item_has_contract,
        'has_contract': has_contract,
        'is_free_text': is_free_text,
        'free_text_materials': free_text_materials,

        'has_qty_change': has_qty_change,

        'has_payment_term_change': random.random() < values.proc_vendors[vendor_name]['payment_term_stsability'],
        'delivery_status': delivery_status,

        'new_payment_term': 'Z090',
        'new_vendor': new_lifnr,
        
        'pr_req_year': pr_req_year,
        'pr_req_month': pr_req_month,
    }

    return params

In [7]:
# x = []
# # y = []

# for i in range(10_000):
#     params = get_params()
#     a = params['prices']
#     b = params['quantities']
#     total = sum([a[j]*b[j] for j in range(len(a))])
#     x.append(total)
#     # y.append(datetime(pr_req_year, pr_req_month, random.randint(1, 12)).month)

In [8]:
# np_x = np.array(x)
# np.quantile(np_x, 0.8)

In [9]:
# from collections import Counter

In [10]:
# Counter(y)

In [11]:
purchasing_doc_tables = {
    'EBAN_json': {}, 
	'CDHDR_json': {},
	'CDPOS_json': {},
	'EKKO_json': {},
	'EKPO_json': {},
	'NAST_json': {},
	'MSEG_json': {},
	'EKBE_json': {},
	'RBKP_json': {},
	'RSEG_json': {},
	'EKET_json': {},
}
for i in range(1_000):
# for i in range(1_000):
	transition_prob = np.array([
		[0.00, 0.50, 1.30, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # Create PO
		[0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10], # Approve PO
		[0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.10], # Send PO
		[0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.01], # Post Goods Receipt
		[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # Create Vendor Invoice

		[0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.05], # Change Payment Term
		[0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.15], # Change Vendor
		[0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.05], # Change Quantity

		[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00] # Terminate
	])
	"""
	0: Create PO
	1: Approve PO
	2: Send PO
	3: Post GR
	4: Create Vendor Invoice

	5: Change Payment Term
	6: Change Vendor
	7: Change Quantity
	
	8: Terminate
	"""

	# Create Contract
	# Create PR
	
	params = get_params()
	latest_date = datetime(params['pr_req_year'], params['pr_req_month'], random.randint(1, 28)).date()
	latest_time = helpers.generate_random_time()
	purchasing = purchasing_doc_data.Purchasing(params=params, start_date=latest_date, index=i)
	preq_creation_date = latest_date

	purchasing.create_contract(
		aedat=latest_date,
		ernam=get_user_name(0.3)
	)
	latest_date += helpers.UPTO_WEEK()

	pr_automation_rate = 0.2 if params['is_free_text'] else 0.7
	purchasing.create_purchase_requisition_item(
		badat=latest_date, 
		ernam=get_user_name(pr_automation_rate),
	)

	step = 0 # Create PO

	purchasing.create_purchase_order(
	aedat=latest_date,
	ernam=get_user_name(0.8)
	)
	latest_date += helpers.UPTO_DAY()
	total = sum([params['quantities'][j]*params['prices'][j] for j in range(len(params['matnrs']))])
	if total > 25_000:
		transition_prob[step][1] = 4.0



	while step != 8:
		if step == 0:
			if (params['is_free_text']) or (not params['has_contract']):
				if random.random() < 0.3:
					step = 6
					latest_date += helpers.UPTO_WEEK() 
					latest_date += helpers.UPTO_WEEK() 
				else:
					latest_date += helpers.UPTO_WEEK() 

		elif step == 1:
			purchasing.approve_purchase_order(
				aedat=latest_date,
				ernam = get_user_name(0.2)
			)
			latest_date += helpers.UPTO_WEEK()

		elif step == 2:
			purchasing.send_purchase_order(
				usnam=get_user_name(0.9),
				erdat=latest_date
			)
			latest_date += helpers.UPTO_DAY()

			if params['has_payment_term_change']:
				transition_prob[step][5] = 1.0

			if params['has_qty_change']: # 50% chance of Change Quantity
				transition_prob[step][7] = 1.0

		elif step == 3:
			purchasing.goods_receipt(
				cpudt=latest_date,
				usnam=get_user_name(0.7),
				atime=latest_time
			)
			latest_date += helpers.UPTO_WEEK()

		elif step == 4:
			purchasing.create_vendor_invoice(
				cupdt=latest_date,
				ernam=get_user_name(0.9),
			)

		elif step == 5:
			purchasing.change_payment_term( # likely goes next to 'Post GR'
				udate=latest_date,
				ernam=get_user_name(0.2)
			)
			latest_date += helpers.UPTO_MONTH()

		elif step == 6:
			purchasing.change_vendor( # likely goes to 'Approve PO'
				udate=latest_date,
				ernam=get_user_name(0.2)
			)
			latest_date += helpers.UPTO_WEEK()

		elif step == 7:
			old_quantities = params['quantities']
			new_quantity_lines = random.sample(range(len(old_quantities)), random.randint(0, len(old_quantities)))
			new_quanity_quantities = [old_quantities[j]*(1+random.random()) for j in new_quantity_lines]
			purchasing.change_quantity( # likely goes to 'Send PO'
				badat=latest_date,
				ernam=get_user_name(0.1),
				line_numbers=new_quantity_lines,
				line_quantities=new_quanity_quantities
			)
			latest_date += helpers.UPTO_WEEK()
		
		# normalize transition matrix
		transition_prob = transition_prob / transition_prob.sum(axis=1, keepdims=True)
		step = np.random.choice(9, p=transition_prob[step])

	for k, v in purchasing.tables.items():
		for entry_key in list(v.keys()):
			purchasing_doc_tables[k][entry_key] = purchasing.tables[k][entry_key]

In [12]:
for table, rows in purchasing_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    all_cols_req = ctl.clean_columns(all_cols,table_name)
    df = pd.concat([all_cols_req, pd.DataFrame(rows.values())])
    df.to_csv(f'data/P2P/OCPM/purchasing-document/{table_name}.csv', index=False)

C:\Users\w.falls\AppData\Local\Temp\ipykernel_2088\646703889.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([all_cols_req, pd.DataFrame(rows.values())])
C:\Users\w.falls\AppData\Local\Temp\ipykernel_2088\646703889.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([all_cols_req, pd.DataFrame(rows.values())])
C:\Users\w.falls\AppData\Local\Temp\ipykernel_2088\646703889.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is 